# Projeto 03 - Converse com documentos usando RAG avançada

> Nesse projeto iremos aprender a como criar uma pipeline de RAG mais avançada para que seja capaz de:
* fazer perguntas a algum documento lido, como se fosse um chat com o próprio arquivo.
* consultar mais de uma referência ao mesmo tempo.
* entender o contexto das mensagens passadas, usando o histórico da conversa também como uma referência para formular a resposta

E para essa aplicação também será construída uma interface.
Portanto, podemos reaproveitar parte do código do projeto anterior e assim ir adicionando as novas funcionalidades

## [ ! ] Como executar em ambiente local
Para executar o código desse projeto em um ambiente local, siga as instruções para instalar as dependências necessárias usando os comandos abaixo. Você pode usar os mesmos comandos de instalação.



## Instalação e Configuração

Aqui iremos carregar primeiramente todos as funções que usamos no projeto anterior e mais algumas outras. Entra elas, o FAISS (um vectorstore no mesmo estilo do Chroma, que usamos nas aulas anteriores sobre RAG) e também outras funções necessárias para implementação de uma pipeline RAG que entenda o contexto das conversas.

Lembrando: podemos reaproveitar parte do código que criamos no projeto 02.
Então se quiser pode fazer uma cópia e fazer as modificações a partir dele.

Abaixo, cada mudança que será feita a partir desse arquivo, assim poderá ir acompanhando as alterações

In [ ]:
!pip install -q streamlit langchain
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 722.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00


### Instalação do FAISS

Antes de importar é necessário que instalemos o FAISS, ele não vem instalado por padrão no Colab. Portanto, podemos usar aqui e em ambiente local esse mesmo comando para instalar:

`pip install -q faiss-cpu`

você também pode instalar `faiss-gpu` caso queira usar a versão otimizada para GPU. Para simplificar no momento, iremos usar a versão padrão por CPU mesmo



In [ ]:
!pip install -q faiss-cpu

Em seguida, use em sua aplicação o `import faiss` e também importar o `FAISS` dentro de biblioteca langchain

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS

### Instalação do PyPDFLoader

Usaremos o PyPDFLoader para fazer a leitura dos arquivos PDF em nossa aplicação. Isso será explicado com detalhes na devida seção.
E para podermos usá-lo, precisamos antes instalar a biblioteca com o comando abaixo

In [ ]:
!pip install pypdf

### Demais instalações

Assim como no projeto anterior, vamos instalar aqui o dotenv de novo (em ambiente local não precisa executar a instalação de novo, mas aqui no Colab como é uma nova sessão precisamos) e também o localtunnel (lembrando que esse não é necessário em ambiente local).

In [ ]:
!pip install -q python-dotenv
!npm install -q localtunnel

In [ ]:
%%writefile .env
HUGGINGFACE_API_KEY=##########
HUGGINGFACEHUB_API_TOKEN=##########
OPENAI_API_KEY=##########
TAVILY_API_KEY=##########
SERPAPI_API_KEY=##########
LANGCHAIN_API_KEY=##########

Writing .env


---

## Inicialização da interface

Por fim, reunimos todo o código em um único script e adicionamos a configuração da página com st.set_page_config e st.title, alterando o título e o emoji para deixar a interface mais personalizada e adequada ao projeto atual, com um visual mais alinhado com o contexto desse projeto

In [ ]:
%%writefile projeto3.py

import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder

from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

import torch
from langchain_huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

import faiss
import tempfile
import os
import time
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader

from dotenv import load_dotenv

load_dotenv()

# Configurações do Streamlit
st.set_page_config(page_title="Converse com documentos 📚", page_icon="📚")
st.title("Converse com documentos 📚")

model_class = "hf_hub" # @param ["hf_hub", "openai", "ollama"]

## Provedores de modelos
def model_hf_hub(model="meta-llama/Meta-Llama-3-8B-Instruct", temperature=0.1):
  llm = HuggingFaceHub(
      repo_id=model,
      model_kwargs={
          "temperature": temperature,
          "return_full_text": False,
          "max_new_tokens": 512,
          #"stop": ["<|eot_id|>"],
          # demais parâmetros que desejar
      }
  )
  return llm

def model_openai(model="gpt-4o-mini", temperature=0.1):
    llm = ChatOpenAI(
        model=model,
        temperature=temperature
        # demais parâmetros que desejar
    )
    return llm

def model_ollama(model="phi3", temperature=0.1):
    llm = ChatOllama(
        model=model,
        temperature=temperature,
    )
    return llm


## Indexação e Recuperação

def config_retriever(uploads):
    # Carregar documentos
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploads:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyPDFLoader(temp_filepath)
        docs.extend(loader.load())

    # Divisão em pedaços de texto / Split
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(docs)

    # Embeddings
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

    # Armazenamento
    vectorstore = FAISS.from_documents(splits, embeddings)

    vectorstore.save_local('vectorstore/db_faiss')

    # Configurando o recuperador de texto / Retriever
    retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k':3, 'fetch_k':4}
    )

    return retriever


def config_rag_chain(model_class, retriever):

    ### Carregamento da LLM
    if model_class == "hf_hub":
        llm = model_hf_hub()
    elif model_class == "openai":
        llm = model_openai()
    elif model_class == "ollama":
        llm = model_ollama()

    # Para definição dos prompts
    if model_class.startswith("hf"):
        token_s, token_e = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>", "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    else:
        token_s, token_e = "", ""

    # Prompt de contextualização
    context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

    context_q_system_prompt = token_s + context_q_system_prompt
    context_q_user_prompt = "Question: {input}" + token_e
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", context_q_user_prompt),
        ]
    )

    # Chain para contextualização
    history_aware_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_q_prompt
    )

    # Prompt para perguntas e respostas (Q&A)
    qa_prompt_template = """Você é um assistente virtual prestativo e está respondendo perguntas gerais.
    Use os seguintes pedaços de contexto recuperado para responder à pergunta.
    Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
    Responda em português. \n\n
    Pergunta: {input} \n
    Contexto: {context}"""

    qa_prompt = PromptTemplate.from_template(token_s + qa_prompt_template + token_e)

    # Configurar LLM e Chain para perguntas e respostas (Q&A)

    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(
        history_aware_retriever,
        qa_chain,
    )

    return rag_chain


## Cria painel lateral na interface
uploads = st.sidebar.file_uploader(
    label="Enviar arquivos", type=["pdf"],
    accept_multiple_files=True
)
if not uploads:
    st.info("Por favor, envie algum arquivo para continuar!")
    st.stop()


if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        AIMessage(content="Olá, sou o seu assistente virtual! Como posso ajudar você?"),
    ]

if "docs_list" not in st.session_state:
    st.session_state.docs_list = None

if "retriever" not in st.session_state:
    st.session_state.retriever = None

for message in st.session_state.chat_history:
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.write(message.content)
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.write(message.content)

# para gravar quanto tempo levou para a geração
start = time.time()
user_query = st.chat_input("Digite sua mensagem aqui...")

if user_query is not None and user_query != "" and uploads is not None:

    st.session_state.chat_history.append(HumanMessage(content=user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):

        if st.session_state.docs_list != uploads:
            print(uploads)
            st.session_state.docs_list = uploads
            st.session_state.retriever = config_retriever(uploads)

        rag_chain = config_rag_chain(model_class, st.session_state.retriever)

        result = rag_chain.invoke({"input": user_query, "chat_history": st.session_state.chat_history})

        resp = result['answer']
        st.write(resp)

        # mostrar a fonte
        sources = result['context']
        for idx, doc in enumerate(sources):
            source = doc.metadata['source']
            file = os.path.basename(source)
            page = doc.metadata.get('page', 'Página não especificada')

            ref = f":link: Fonte {idx}: *{file} - p. {page}*"
            print(ref)
            with st.popover(ref):
                st.caption(doc.page_content)

    st.session_state.chat_history.append(AIMessage(content=resp))

end = time.time()
print("Tempo: ", end - start)

Writing projeto3.py


### Execução do Streamlit




In [ ]:
!streamlit run projeto3.py &>/content/logs.txt &

E agora para nos conectar, usamos o comando abaixo (explicações no colab do projeto 02)

In [ ]:
!wget -q -O - ipv4.icanhazip.com

!npx localtunnel --port 8501

34.16.159.1
⠙your url is: https://short-lizards-hunt.loca.lt


## Como melhorar 🚀

Sabendo exatamente como cada função opera e entendendo as explicações tratadas ao longo deste projeto, você tem o conhecimento necessário para melhorar os resultados da sua aplicação RAG. Vamos listar abaixo as estratégias que podem ser aplicadas para otimizar a qualidade das respostas e a eficiência do sistema:

* Testar outros modelos de Embedding - conforme citado, a seleção do modelo correto para o sistema RAG é crucial, pois afeta diretamente a precisão das respostas, além da utilização de recursos e a escalabilidade da aplicação. Escolher modelos que funcionem bem com o idioma e a tarefa em questão pode melhorar significativamente os resultados. Ao testar diferentes modelos, você pode identificar qual oferece a melhor combinação de qualidade e eficiência para as suas necessidades.

* Ajustar o prompt fixo (do sistema) - Modificar o prompt do sistema para torná-lo mais explícito sobre as funções que a LLM deve desempenhar pode melhorar os resultados. O prompt deve especificar com clareza o que a LLM deve priorizar na resposta e o que deve ser ignorado. Isso orienta o modelo a focar no que é mais relevante para sua aplicação e seu objetivo.

* Melhorar o prompt do usuário - lembrar o usuário (colocando um aviso na interface talvez) que quanto mais específico for na pergunta maior a chance de aumentar a precisão das respostas geradas pela LLM. Quanto mais detalhado e claro o pedido, mais relevante será o retorno. Esta prática também ajuda a reduzir ambiguidades que podem prejudicar a interpretação da consulta pelo modelo.

* Ajustar o prompt de contextualização - lembrando que este prompt reformula a pergunta do usuário com base no histórico da conversa, algo útil quando a consulta precisa de contexto para ser corretamente interpretada. O prompt de contextualização (context_q_system_prompt) instrui o modelo a levar o histórico em consideração; e embora o prompt atual esteja em inglês devido à maior chance de compatibilidade da LLM com este idioma (apesar de ser compatível com o nosso), você pode testá-lo em português e assim fica fácil modificar o texto para maximizar o desempenho no idioma desejado.

* Testar outras LLMs - Explorar outros modelos de linguagem, especialmente aqueles que aceitam uma quantidade maior de tokens e têm bom desempenho no idioma escolhido, pode melhorar a performance. Para casos mais exigentes, pode valer a pena considerar soluções proprietárias como o ChatGPT ou serviços pagos (como o Groq, citado no Colab 1) que disponibilizam grandes modelos de código aberto. Modelos maiores podem lidar melhor com consultas complexas e fornecer respostas mais elaboradas.

* Ajustar os parâmetros de recuperação (k e fetch_k) - Modificar os parâmetros das etapas de recuperação, como os valores de k e fetch_k, pode ter um impacto significativo no desempenho da sua aplicação. Experimente começar com valores menores e aumentá-los conforme necessário, sempre monitorando o impacto na relevância e qualidade das respostas. Para mais detalhes, consulte a seção da pipeline RAG e o retriever. Outra ideia seria testar outros algoritmos além do MMR.

* Deixar melhor preparado para aceitar qualquer documento - uma ideia é fazer o preprocessamento de arquivos PDF (ou outros formatos) para adequação ao vector store. Muitas vezes PDFs possuem tabelas ou outras estruturas que dificultam a interpretação; ou ainda, documentos em formatos mais diferentes como HTML, CSV, ou PPTX não estão estruturados para extração ideal de informações. A preparação desses arquivos é crucial para garantir que o conteúdo relevante seja corretamente capturado e disponibilizado para o sistema de recuperação.
 * Existem soluções especializadas automatizam essa transformação, organizando os dados e eliminando informações desnecessárias. Isso otimiza o fluxo de trabalho e melhora a precisão dos resultados. Um exemplo é o serviço Unstructured (Acesse https://unstructured.io), que facilita a extração de dados complexos de arquivos, tornando-os prontos para uso em bancos de dados vetoriais e frameworks de LLMs, o que aumenta a qualidade da recuperação da informação e o desempenho da aplicação RAG.
 * Para usar isso no langchain é simples, você pode usar o método de Document Loader. Na prática, basta carregar o documento usando o document loader Unstructured (ao invés do PyPDFLoader que usamos). Mais detalhes aqui: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file/


Essas estratégias visam otimizar a eficiência e a qualidade das respostas do sistema RAG, adaptando-o ao seu caso de uso específico.


